In [1]:
# Operações de Crédito - Conta 16000001 (filtragem maiores saldos)

import pandas as pd
import sqlite3

conn = sqlite3.connect('dados/banking.db')

query = """
WITH RankedData AS (
    SELECT
        *,
        ROW_NUMBER() OVER(PARTITION BY data ORDER BY "SALDO" DESC) AS rn
    FROM
        balancetes_bancos
    WHERE
        "CONTA" = '16000001'
)
SELECT
    *
FROM
    RankedData
WHERE
    SALDO > 1000000000
--    rn <= 10;
"""

model = pd.read_sql_query(query, conn)

conn.close()

model = model[['data', 'cnpj', 'NOME_INSTITUICAO', 'SALDO']]
model['cnpj'] = model['cnpj'].astype(int)
model = model.rename(columns={'SALDO': 'operacoes_de_credito'})


In [2]:
# Passivo Total -  Conta 99999995

import numpy as np

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    *
FROM
    balancetes
WHERE
    "CONTA" = '99999995'
"""

table = pd.read_sql_query(query, conn)

conn.close()

table = table[['data', 'cnpj', 'SALDO']]
table['cnpj'] = (table['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
table.dropna(subset=['cnpj'], inplace=True)
table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'SALDO': 'passivo_total'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)


In [3]:
# Patrimônio Líquido - Conta 60000002

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    *
FROM
    balancetes
WHERE
    "CONTA" = '60000002'
"""

table = pd.read_sql_query(query, conn)

conn.close()

table = table[['data', 'cnpj', 'SALDO']]
table['cnpj'] = (table['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
table.dropna(subset=['cnpj'], inplace=True)
table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'SALDO': 'patrimonio_liquido'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)


In [4]:
# Alavancagem (lever)

model['lever'] = model['passivo_total'] / model['patrimonio_liquido']

In [5]:
# Fundos Remunerados
'''
Depósitos de Poupança (4.1.2.00.00-3)
Depósitos Interfinanceiros (4.1.3.00.00-6)
Depósitos a Prazo (4.1.5.00.00-2)
Depósitos sob Aviso (4.1.4.00.00-9)
Obrigações por Operações Compromissadas (4.2.0.00.00-6)
Obrigações por Títulos e Valores Mobiliários (4.3.0.00.00-5)
Obrigações por Empréstimos e Repasses (4.6.0.00.00-2)
'''

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('41200003', '41300006', '41500002', '41400009', '42000006', '43000005', '46000002')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'fundos_remunerados'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)


In [6]:
# Ativos rentáveis
'''
Aplicações Interfinanceiras de Liquidez (1.2.0.00.00-5)
Títulos e Valores Mobiliários e Instrumentos Financeiros Derivativos (1.3.0.00.00-4)
Operações de Crédito (1.6.0.00.00-1)
Operações de Arrendamento Mercantil (1.7.0.00.00-0)
Outros Créditos (1.8.0.00.00-9)
'''

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('12000005', '13000004', '16000001', '17000000', '18000009')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = (table['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
table.dropna(subset=['cnpj'], inplace=True)
table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'ativos_rentaveis'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)


In [7]:
# ibf (Interest Bearing Funds / Total Earning Assets)

model['ibf'] = model['fundos_remunerados'] / model['ativos_rentaveis']

In [8]:
# Depósitos à Vista (Depósitos Não Remunerados) - Conta 41100000
'''
Depósitos à Vista (4.1.1.00.00-0)
'''

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    *
FROM
    balancetes
WHERE
    "CONTA" = '41100000'
"""

table = pd.read_sql_query(query, conn)

conn.close()

table = table[['data', 'cnpj', 'SALDO']]
table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'SALDO': 'depositos_a_vista'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)



In [9]:
# Depósitos - Conta 41000007

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    *
FROM
    balancetes
WHERE
    "CONTA" = '41000007'
"""

table = pd.read_sql_query(query, conn)

conn.close()

table = table[['data', 'cnpj', 'SALDO']]
table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'SALDO': 'depositos'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)


In [10]:
# Non-Interest Bearing Deposits (nibd) = Depósitos à vista (Não Remunerados) / Depósitos Totais

model['nibd'] = model['depositos_a_vista'] / model['depositos']

In [11]:
# Custos Operacionais
'''
Despesas de Captação (8.1.1.00.00-8)
Despesas de Obrigações por Empréstimos e Repasses (8.1.2.00.00-1)
Despesas de Arrendamento Mercantil (8.1.3.00.00-4)
Despesas de Câmbio (8.1.4.00.00-7)
Despesas com Títulos e Valores Mobiliários e Instrumentos Financeiros Derivativos (8.1.5.00.00-0)
Despesas Administrativas (8.1.7.00.00-6)
Despesas de Provisões Operacionais (8.1.8.30.00-0)
Outras Despesas Operacionais (8.1.9.99.00-6)
'''

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('81100008', '81200001', '81300004', '81400007', '81500000', '81700006', '81830000', '81999006')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'custos_operacionais'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)


In [12]:
# Ativo Total - Conta 39999993

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    *
FROM
    balancetes
WHERE
    "CONTA" = '39999993'
"""

table = pd.read_sql_query(query, conn)

conn.close()

table = table[['data', 'cnpj', 'SALDO']]
table['cnpj'] = (table['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
table.dropna(subset=['cnpj'], inplace=True)
table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'SALDO': 'ativo_total'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)


In [13]:
# Custos Operacionais sobre o Ativo (opc)

model['opc'] = model['custos_operacionais'] / model['ativo_total']

In [14]:
# Ativos de Alta Liquidez (Caixa e Equivalentes de Caixa)
'''
Caixa (1.1.1.00.00-9)
Depósitos Bancários (1.1.2.00.00-2)
Reservas Livres (1.1.3.00.00-5)
Aplicações em Operações Compromissadas (1.2.1.00.00-8)
Aplicações em Depósitos Interfinanceiros (1.2.2.00.00-1)
'''

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('11100009', '11200002', '11300005', '12100008', '12200001')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'ativo_liquido'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)


In [15]:
# Passivos de Curto Prazo (Obrigações Exigíveis no Circulante)
'''
Depósitos à Vista (4.1.1.00.00-0)
Obrigações por Operações Compromissadas (4.2.0.00.00-6)
Depósitos Interfinanceiros (4.1.3.00.00-6)
'''

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('41100000', '42000006', '41300006')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'passivo_curto_prazo'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)

In [16]:
# Liquidez (liquid)

model['liquid'] = model['ativo_liquido'] / model['passivo_curto_prazo']

In [17]:
# Renda de Serviços - Conta 71700009

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    *
FROM
    balancetes
WHERE
    "CONTA" = '71700009'
"""

table = pd.read_sql_query(query, conn)

conn.close()

table = table[['data', 'cnpj', 'SALDO']]
table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'SALDO': 'renda_servicos'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)

In [18]:
# Receita Operacional - Conta 71000008

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    *
FROM
    balancetes
WHERE
    "CONTA" = '71000008'
"""

table = pd.read_sql_query(query, conn)

conn.close()

table = table[['data', 'cnpj', 'SALDO']]
table['cnpj'] = (table['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
table.dropna(subset=['cnpj'], inplace=True)
table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'SALDO': 'receita_operacional'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)

In [19]:
# razão entre receitas de serviços e o total de receitas operacionais (servr)

model['servr'] = model['renda_servicos'] / model['receita_operacional']

In [20]:
# Ativos liquidos não remunerados
'''
Caixa (1.1.1.00.00-9)
Reservas Livres (1.1.3.00.00-3)
Depósitos Bancários (1.1.2.00.00-6)
'''
'''
conn = sqlite3.connect('dados/banking.db')
query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('11100009', '11900003', '11200006')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'ativos_liquidos_nao_remunerados'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)
'''

'\nconn = sqlite3.connect(\'dados/banking.db\')\nquery = """\nSELECT\n    data,\n    cnpj,\n    SUM("SALDO") AS saldo_total_agregado\nFROM\n    balancetes\nWHERE\n    "CONTA" IN (\'11100009\', \'11900003\', \'11200006\')\nGROUP BY\n    data,\n    cnpj\nORDER BY\n    data,\n    cnpj;\n"""\n\ntable = pd.read_sql_query(query, conn)\nconn.close()\n\ntable[\'cnpj\'] = table[\'cnpj\'].astype(int)\ntable = table.rename(columns={\'saldo_total_agregado\': \'ativos_liquidos_nao_remunerados\'})\n\nmodel = pd.merge(\n    model,\n    table,\n    on=[\'data\', \'cnpj\'],\n    how=\'left\'\n)\n'

In [21]:
# Receitas de Juros e Similares (Rendas)
'''
Rendas De Operacoes De Credito (7.1.1.00.00-1)
Rendas de Arrendamento Mercantil (7.1.2.00.00-4)
Rendas de Aplicações Interfinanceiras de Liquidez (7.1.4.00.00-0)
Rendas de Títulos e Valores Mobiliários (7.1.5.00.00-3)
Rendas de Outras Operações Com Características de Crédito (7.1.6.00.00-8)
'''
conn = sqlite3.connect('dados/banking.db')
query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('71100001', '71200004', '71400000', '71500003', '71600008')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'receita_juros'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)


In [22]:
# Despesas de Juros e Encargos (Custos de Captação)
'''
Despesas de captação (81100008)
Despesas de Obrigações por Empréstimos e Repasses (8.1.2.00.00-1)
'''
conn = sqlite3.connect('dados/banking.db')
query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('81100008', '81200001')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'despesa_captacao'})
table['despesa_captacao'] = table['despesa_captacao'] * (-1)

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)


In [23]:
# Ativos Remunerados Médios (Média do Balanço)
'''
Operações de Crédito (1.6.0.00.00-1)
Títulos e Valores Mobiliários (1.3.0.00.00-4)
Arrendamento Mercantil (1.7.0.00.00-1)
'''
conn = sqlite3.connect('dados/banking.db')
query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('16000001', '13000004', '17000001')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = (table['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
table.dropna(subset=['cnpj'], inplace=True)
table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'ativos_remunerados'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)


In [24]:
# spread = taxa_emprestimos - taxa_captacao

model['spread'] = (model['receita_juros'] - model['despesa_captacao']) / model['ativos_remunerados']

In [25]:
variables = model[['data', 'cnpj', 'NOME_INSTITUICAO', 'spread', 'nibd', 'ibf', 'lever', 'opc', 'liquid', 'servr']]


In [26]:
# Adicionando variáveis lags

coluna_grupo = 'cnpj'
colunas_para_verificar = ['nibd', 'ibf', 'lever', 'opc', 'liquid', 'servr']

for col in colunas_para_verificar:

    variables[f'{col}_lag_2'] = (variables.groupby(coluna_grupo)[col].shift(2))
    variables[f'{col}_lag_1'] = (variables.groupby(coluna_grupo)[col].shift(1))
    variables[f'{col}_lag_3'] = (variables.groupby(coluna_grupo)[col].shift(3))


/tmp/ipykernel_1064108/254897898.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  variables[f'{col}_lag_2'] = (variables.groupby(coluna_grupo)[col].shift(2))
/tmp/ipykernel_1064108/254897898.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  variables[f'{col}_lag_1'] = (variables.groupby(coluna_grupo)[col].shift(1))
/tmp/ipykernel_1064108/254897898.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

In [27]:
# Padronização da data e remoção de NaNs

todas_colunas = variables.columns.tolist()
colunas_para_verificar = [col for col in todas_colunas if col != 'NOME_INSTITUICAO']
variables = variables.dropna(how='any', subset=colunas_para_verificar)
#variables['data'] = pd.to_datetime(variables['data'], format='%Y-%m').dt.strftime('%Y-%m')

In [28]:
# Adicionando Dummies de datas

dummies_tempo = pd.get_dummies(variables['data'], drop_first=True)
dummies_tempo = dummies_tempo.astype(int)
variables = pd.concat([variables, dummies_tempo], axis=1)

In [31]:
# Modelo - Primeiro Estágio
import statsmodels.api as sm

colunas_X_originais = ['nibd', 'ibf', 'lever', 'opc', 'liquid', 'servr', 'nibd_lag_1', 'nibd_lag_2', 'nibd_lag_3', 'ibf_lag_1', 'ibf_lag_2', 'ibf_lag_3', 'lever_lag_1', 'lever_lag_2', 'lever_lag_3', 'opc_lag_1', 'opc_lag_2', 'opc_lag_3', 'liquid_lag_1', 'liquid_lag_2', 'liquid_lag_3', 'servr_lag_1', 'servr_lag_2', 'servr_lag_3']
colunas_X = colunas_X_originais + list(dummies_tempo.columns)
X = variables[colunas_X]
y = variables['spread']

stage1 = sm.OLS(y, sm.add_constant(X)).fit()
results_stage1 = stage1
print(results_stage1.summary())

                            OLS Regression Results                            
Dep. Variable:                 spread   R-squared:                       0.796
Model:                            OLS   Adj. R-squared:                  0.792
Method:                 Least Squares   F-statistic:                     178.2
Date:                Sat, 25 Oct 2025   Prob (F-statistic):               0.00
Time:                        14:46:05   Log-Likelihood:                -690.46
No. Observations:                9319   AIC:                             1783.
Df Residuals:                    9118   BIC:                             3218.
Df Model:                         200                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.0420      0.044      0.952   